In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(48,48,3))
    base_model.trainable=True
    input_layer = Input(shape=(48,48,3))
    feat1 = GlobalAveragePooling2D()(input_layer)
    x = base_model.layers[1](input_layer)
    x = base_model.layers[2](x)
    x = base_model.layers[3](x)
    feat2 = GlobalAveragePooling2D()(x)
    x = base_model.layers[4](x)
    x = base_model.layers[5](x)
    x = base_model.layers[6](x)
    feat3 = GlobalAveragePooling2D()(x)
    x = base_model.layers[7](x)
    x = base_model.layers[8](x)
    x = base_model.layers[9](x)
    x = base_model.layers[10](x)
    feat4 = GlobalAveragePooling2D()(x)
    x = base_model.layers[11](x)
    x = base_model.layers[12](x)
    x = base_model.layers[13](x)
    x = base_model.layers[14](x)
    feat5 = GlobalAveragePooling2D()(x)
    x = base_model.layers[15](x)
    x = base_model.layers[16](x)
    x = base_model.layers[17](x)
    x = base_model.layers[18](x)
    feat6 = GlobalAveragePooling2D()(x)
    x = tf.concat([feat1, feat2, feat3, feat4, feat5, feat6], -1)
    x = Dense(units=4096, activation='relu')(x)
    x = Dense(units=4096, activation='relu')(x)
    output_layer = Dense(units=8, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 2e-5
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()

model = create_model()
history_save_path = './results/combined.txt'
history = train(model, learning_rate, loss, num_epochs, batch_size)

with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()

with open('./results/history_base+flip+rotate20.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'vgg16')

with open('./results/combined.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'combined')

plt.legend()
plt.show()
# save plot
fig.savefig('./plots/combined.png')